In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp lag_transforms

# Lag transforms
> Built-in lag transformations

In [ ]:
#| export
from typing import Optional

import numpy as np

from mlforecast.compat import CoreGroupedArray, core_tfms

In [ ]:
#| exporti
class BaseLagTransform:
    _core_tfm: core_tfms.BaseLagTransform

    def transform(self, ga: CoreGroupedArray) -> np.ndarray:
        return self._core_tfm.transform(ga)

    def update(self, ga: CoreGroupedArray) -> np.ndarray:
        return self._core_tfm.update(ga)

In [ ]:
#| export
class Lag(BaseLagTransform):
    def __init__(self, lag: int):
        self.lag = lag
        self._core_tfm = core_tfms.Lag(lag=lag)

    def __eq__(self, other):
        return isinstance(other, Lag) and self.lag == other.lag

In [ ]:
#| exporti
class RollingBase(BaseLagTransform):
    def __init__(self, window_size: int, min_samples: Optional[int] = None):
        self.window_size = window_size
        self.min_samples = min_samples

    def _set_core_tfm(self, lag: int):
        self._core_tfm = getattr(core_tfms, self.tfm_name)(lag=lag, window_size=self.window_size, min_samples=self.min_samples)
        return self

In [ ]:
#| export
class RollingMean(RollingBase):
    tfm_name = 'RollingMean'

class RollingStd(RollingBase):
    tfm_name = 'Rollingstd'


class RollingMin(RollingBase):
    tfm_name = "RollingMin"


class RollingMax(RollingBase):
    tfm_name = "RollingMax"

In [ ]:
#| core
lengths = np.random.randint(low=50, high=100, size=20)
data = np.random.rand(lengths.sum())
ga = CoreGroupedArray(data, np.append(0, lengths.cumsum()))
RollingMean(7)._set_core_tfm(1).transform(ga)

array([       nan,        nan,        nan, ..., 0.49484379, 0.52374839,
       0.55126774])

In [ ]:
#| exporti
class SeasonalRollingBase(BaseLagTransform):
    def __init__(
        self, season_length: int, window_size: int, min_samples: Optional[int] = None
    ):
        self.season_length = season_length
        self.window_size = window_size
        self.min_samples = min_samples

    def _set_core_tfm(self, lag: int):
        self._core_tfm = getattr(core_tfms, self.tfm_name)(
            lag=lag, season_length=self.season_length, window_size=self.window_size, min_samples=self.min_samples
        )
        return self

In [ ]:
# | export
class SeasonalRollingMean(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingMean'

class SeasonalRollingStd(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingStd'

class SeasonalRollingMin(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingMin'

class SeasonalRollingMax(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingMax'

In [ ]:
#| core
SeasonalRollingStd(7, 4)._set_core_tfm(2).transform(ga)

array([       nan,        nan,        nan, ..., 0.37644922, 0.34515196,
       0.23084811])

In [ ]:
#| exporti
class ExpandingBase(BaseLagTransform):
    def __init__(self):
        ...
    
    def _set_core_tfm(self, lag: int):
        self._core_tfm = getattr(core_tfms, self.tfm_name)(lag=lag)
        return self

In [ ]:
#| export
class ExpandingMean(ExpandingBase):
    tfm_name = 'ExpandingMean'

class ExpandingStd(ExpandingBase):
    tfm_name = 'ExpandingStd'

class ExpandingMin(ExpandingBase):
    tfm_name = 'ExpandingMin'

class ExpandingMax(ExpandingBase):
    tfm_name = 'ExpandingMax'

In [ ]:
#| core
ExpandingMin()._set_core_tfm(3).transform(ga)

array([       nan,        nan,        nan, ..., 0.02353498, 0.02353498,
       0.02353498])

In [ ]:
#| export
class ExponentiallyWeightedMean(BaseLagTransform):
    def __init__(self, alpha: float):
        self.alpha = alpha

    def _set_core_tfm(self, lag: int):        
        self._core_tfm = core_tfms.ExponentiallyWeightedMean(lag=lag, alpha=self.alpha)
        return self

In [ ]:
#| core
ExponentiallyWeightedMean(0.7)._set_core_tfm(4).transform(ga)

array([       nan,        nan,        nan, ..., 0.37994511, 0.16175122,
       0.68870614])